In [1]:
import zipline
# from zipline.examples import buyapple

%load_ext zipline

In [2]:
%%zipline --start 2014-1-1 --end 2018-1-1 -o data/dma.pickle --capital-base 1000


from zipline.api import order, order_target, record, symbol, order_value
import matplotlib.pyplot as plt
from yahoofinancials import YahooFinancials

# For this example, we're going to write a simple momentum script.  
# When the stock goes up quickly, we're going to buy; 
# when it goes down we're going to sell.  
# Hopefully we'll ride the waves.

# To run an algorithm in Quantopian, you need two functions: 
# initialize and handle_data.
def initialize(context):
    # The initialize function sets any data or variables that 
    # you'll use in your algorithm. 
    # For instance, you'll want to define the security 
    # (or securities) you want to backtest.  
    # You'll also want to define any parameters or values 
    # you're going to use later. 
    # It's only called once at the beginning of your algorithm.
    
    # In our example, we're looking at Apple.  
    # If you re-type this line you'll see 
    # the auto-complete that is available for security. 
    context.security = symbol('AAPL')

    # limit the number of companies returned in a fundamentals query
    context.limit = 10

def before_trading_start(context, data):
    context.fundamentals = get_fundamentals(
        query(
            fundamentals.valuation_ratios.pb_ratio,
            fundamentals.valuation_ratios.pe_ratio,
        )
        .filter(
            fundamentals.valuation_ratios.pe_ratio < 14
        )
        .filter(
            fundamentals.valuation_ratios.pb_ratio < 2
        )
        .order_by(
            fundamentals.valuation.market_cap.desc()
        )
        .limit(context.limit)
    )
    
    update_universe(context.fundamentals.columns.values)

# The handle_data function is where the real work is done.  
# This function is run either every minute 
# (in live trading and minute backtesting mode) 
# or every day (in daily backtesting mode).
def handle_data(context, data):
    # We've built a handful of useful data transforms for you to use,
    # such as moving average. 
    
    # To make market decisions, we're calculating the stock's 
    # moving average for the last 5 days and its current price. 
    average_price = data[context.security].mavg(5)
    current_price = data[context.security].price
    MA1 = data[context.security].mavg(50)
    MA2 = data[context.security].mavg(200)
    current_positions = context.portfolio.positions[context.security].amount
    
    # Another powerful built-in feature of the Quantopian backtester is the
    # portfolio object.  The portfolio object tracks your positions, cash,
    # cost basis of specific holdings, and more.  In this line, we calculate
    # the current amount of cash in our portfolio.   
    cash = context.portfolio.cash
    print("cash before trade %s" % (cash))

    for stock in data:
        current_position = context.portfolio.positions[stock].amount
        stock_price = data[stock].price
        plausible_investment = cash / context.limit
     
        share_amount = int(plausible_investment / stock_price)

        try:
                if stock_price < plausible_investment:
                    if current_position == 0:
                        if context.fundamentals[stock]['pe_ratio'] < 11:
                            order(stock, share_amount)
    
        except Exception as e:
            print(str(e))
    
    # Here is the meat of our algorithm.
    # If the current price is 1% above the 5-day average price 
    # AND we have enough cash, then we will order.
    # If the current price is below the average price, 
    # then we want to close our position to 0 shares.

    # if current_price > 1.01*average_price and cash > current_price:
        
    #     # Need to calculate how many shares we can buy
    #     number_of_shares = int(cash/current_price)
        
    #     # Place the buy order (positive means buy, negative means sell)
    #     order(context.security, +number_of_shares)
    #     print("Buying %s %s" % (number_of_shares, context.security.symbol))
        

    # elif current_price < average_price:
        
    #     # Sell all of our shares by setting the target position to zero
    #     order_target(context.security, 0)
    #     print("Selling %s" % (context.security.symbol))
    
    # You can use the record() method to track any custom signal. 
    # The record graph tracks up to five different variables. 
    # Here we record the Apple stock price.
    record(stock_price=data[context.security].price)

NameError: name 'get_fundamentals' is not defined

In [55]:
from iexfinance.refdata import get_symbols
from iexfinance.stocks import Stock, get_collections
import pandas as pd

# symbols=get_symbols(output_format='pandas', token="pk_1dd03915b1bd48c8aa96fa6340d2bab1")

In [328]:
symbols['type']='cs'
symbols_only=symbols[['symbol']]

stockList = symbols_only.head(200).values.flatten().tolist()

In [294]:
shares['peRatio']
shares['sharesOutstanding']
shares.columns
len(stockList)

10

In [335]:
def batchCallApi(symbols, types, filt):
    token='pk_1dd03915b1bd48c8aa96fa6340d2bab1'
    baseUrl = 'https://cloud.iexapis.com/stable/stock/market/batch?'
    route = 'symbols={}&types={}&token={}&filter={}'.format(','.join(symbols), types, token, filt)
    print(baseUrl+route)
    # resp = requests.get(baseUrl+route)
    # if resp.status_code == 200:
    #     return resp.json()
    # print('Response %d - ' % resp.status_code, resp.text)

In [322]:
# out=batchCallApi(stockList,'pk_1dd03915b1bd48c8aa96fa6340d2bab1','stats,book','peRatio, sharesOutstanding')

In [336]:
from multiprocessing.pool import ThreadPool
args = []
empty_data = []
list_orig = empty_data.__class__
types='stats, book'
filt='peRatio, sharesOutstanding'

for i in range(0, len(stockList), 99):
    args.append((stockList[i:i+99], types, filt))

pool = ThreadPool(20)
rets = pool.starmap(batchCallApi, args)
pool.close()

ret = {}

for i, d in enumerate(rets):
    symbols_subset = args[i][0]
    if len(d) != len(symbols_subset):
        empty_data.extend(list_orig(set(symbols_subset) - set(d.keys())))
    ret.update(d)

for k in empty_data:
    if k not in ret:
        if isinstance(fields, str):
            ret[k] = {}
        else:
            ret[k] = {x: {} for x in fields}

ret

https://cloud.iexapis.com/stable/stock/market/batch?symbols=A,AA,AAAU,AACG,AADR,AAL,AAMC,AAME,AAN,AAOI,AAON,AAP,AAPL,AAT,AAU,AAWW,AAXJ,AAXN,AB,ABB,ABBV,ABC,ABCB,ABEO,ABEQ,ABEV,ABG,ABIO,ABM,ABMD,ABR,ABR-A,ABR-B,ABR-C,ABT,ABTX,ABUS,AC,ACA,ACAD,ACAM,ACAMU,ACAMW,ACB,ACBI,ACC,ACCO,ACEL,ACEL+,ACER,ACES,ACGL,ACGLO,ACGLP,ACH,ACHC,ACHV,ACIA,ACIO,ACIU,ACIW,ACLS,ACM,ACMR,ACN,ACNB,ACOR,ACP,ACRE,ACRS,ACRX,ACSG,ACSI,ACST,ACT,ACTG,ACTT,ACTTU,ACTTW,ACU,ACV,ACWF,ACWI,ACWV,ACWX,ACY,ADAP,ADBE,ADC,ADES,ADI,ADIL,ADILW,ADM,ADMA,ADME,ADMP,ADMS,ADNT&types=stats, book&token=pk_1dd03915b1bd48c8aa96fa6340d2bab1&filter=peRatio, sharesOutstanding
https://cloud.iexapis.com/stable/stock/market/batch?symbols=ADP,ADPT,ADRE,ADRO,ADS,ADSK,ADSW,ADT,ADTN,ADUS,ADVM,ADX,ADXN,ADXS,AE,AEB,AEE,AEF,AEFC,AEG,AEGN,AEHR,AEIS,AEL,AEL-A,AEM,AEMD,AEO,AEP,AEP-B,AER,AERI,AES,AESE,AESR,AEY,AEYE,AEZS,AFB,AFC,AFG,AFGB,AFGC,AFGH,AFH,AFI,AFIF,AFIN,AFINP,AFK,AFL,AFLG,AFMC,AFMD,AFSM,AFT,AFTY,AFYA,AG,AGBA,AGBAR,AGBAU,AGBAW,AGCO,AGD,AGE,AGEN,AG

TypeError: object of type 'NoneType' has no len()

In [94]:
import requests
import json
# https://cloud.iexapis.com/stable/stock/market/batch?symbols=[%27AAPL%27,%27TSLA%27]&types=stats&token=pk_1dd03915b1bd48c8aa96fa6340d2bab1&filter=peRatio,%20sharesOutstanding&range=max&last=1
baseUrlGoogle = 'https://cloud.iexapis.com/stable/stock/market/batch?symbols='

searchUrl = baseUrlGoogle + str(stockList) + '&types=stats, book&token=pk_1dd03915b1bd48c8aa96fa6340d2bab1'
searchRes = requests.get(searchUrl)


In [327]:
# import pyEX as p
# c = p.Client(api_token='pk_1dd03915b1bd48c8aa96fa6340d2bab1')
# res=c.bulkBatch(symbols=stockList, fields=['stats','book'])
# c.batch(symbols=['AAPL','TSLA'], fiel)


In [326]:
dfStock=pd.DataFrame(out)
dfStock=dfStock.transpose()
dfStock = pd.concat([pd.DataFrame(x,index=[0]) for x in dfStock['stats']], keys=dfStock.index)
dfStock

,,peRatio,sharesOutstanding
A,0,32.85,310768000
AA,0,-1.96,185915000
AAAU,0,NaN,12100000
AACG,0,-2.12,28020495
AADR,0,NaN,2250000
AAL,0,4.21,426059000
AAMC,0,-13.33,1619000
AAME,0,-27.45,20480000
AAN,0,79.05,66745000
AAOI,0,-2.74,20231000


In [290]:
dfFiltered=dfStock
dfFiltered=dfFiltered.dropna(subset=['peRatio'])
# dfStock.columns()
dfFiltered = dfFiltered[(dfFiltered['peRatio'].astype(float)<15) & (dfFiltered['peRatio'].astype(float)>0)]
dfFiltered['peRatio']
dfFiltered.columns

Index(['avg10Volume', 'avg30Volume', 'beta', 'companyName', 'day200MovingAvg',
       'day30ChangePercent', 'day50MovingAvg', 'day5ChangePercent',
       'dividendYield', 'employees', 'exDividendDate', 'float', 'marketcap',
       'maxChangePercent', 'month1ChangePercent', 'month3ChangePercent',
       'month6ChangePercent', 'nextDividendDate', 'nextEarningsDate',
       'peRatio', 'sharesOutstanding', 'ttmDividendRate', 'ttmEPS',
       'week52change', 'week52high', 'week52low', 'year1ChangePercent',
       'year2ChangePercent', 'year5ChangePercent', 'ytdChangePercent'],
      dtype='object')